In [142]:
import duckdb
import pandas as pd
import duckdb
import numpy as np


In [143]:
def df_to_DBtable(DB,df,table): # repeated
    """
    Creates a persistent table in DuckDB from the DataFrame content.
    """
    con = duckdb.connect(DB)
    con.register(table, df)
    con.execute(f'CREATE TABLE {table} AS SELECT * FROM {table}')
    con.close()

def DBtable_to_df(DB,table): # repeated
    """
    Converts the DB `table` in a data frame format 
    """
    con = duckdb.connect(DB)
    df = con.execute(f'SELECT * FROM {table}').df()
    con.close()
    return df

def get_tables(DB): # repeated
    """
    Gets all the tables from the `DB`
    """
    con = duckdb.connect(DB)
    tables = con.execute(f'SELECT table_name FROM information_schema.tables').df()['table_name']
    con.close()
    return tables

### ETL household table

In [144]:
con = duckdb.connect('household.duckdb')

dfH = con.execute("SELECT *  FROM household").df()
con.close()




dfH['section'] = dfH.section.apply(lambda s: s.strip()[:-2] if s.strip()[-2:]=='.0' else s)
dfH['section'] = dfH.section.apply(lambda s: '0' + s if len(s)==4  else s)
dfH['section'] = dfH.section.apply(lambda s: '0796' + s if len(s)== 5 else s)
dfH['section'] = dfH.section.apply(lambda s: '0796' if s.strip()[-6:]=='Madrid' else s)
dfH['section'] = dfH.section.apply(lambda s: '0796' + s.strip()[0:2] if s.strip()[2]=='.'else s)




dfH['section'] = dfH['section'].str.replace(' ', '') # remove extra spaces


dfH = dfH.rename(columns = {'section':'Madrid_section', 'habitantes':'suma de categorías de hogares'})  #rename in right format

df_to_DBtable(f'integration.duckdb',dfH, 'householdClean_Madrid')

display(dfH)

,Madrid_section,suma de categorías de hogares,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,...,two_adults_over_35_and_one_adult_from_16_to_34_and_two_minors,three_adults_and_0_or_more_minors,two_adults_over_35_and_two_adults_from_16_to_34,two_adults_over_35_and_two_adults_from_16_to_34_and_one_minor,two_adults_over_35_and_two_adults_from_16_to_34_and_two_or_more_minors,four_adults_and_0_or_more_minors,five_adults_and_0_or_more_minors,fifteen_or_more_inhabitants,only_minors,Year
0,0796,1278258.0,116921.0,114806.0,124815.0,35963.0,27460.0,5488.0,156118.0,173809.0,...,8792.0,102020.0,50724.0,7803.0,3361.0,45696.0,50463.0,513.0,342.0,2018
1,079601,66411.0,11695.0,13210.0,5173.0,2142.0,970.0,191.0,12277.0,5164.0,...,191.0,4013.0,1060.0,183.0,79.0,1718.0,1854.0,43.0,18.0,2018
2,079601001,542.0,72.0,86.0,63.0,16.0,10.0,4.0,82.0,63.0,...,0.0,30.0,13.0,2.0,2.0,13.0,18.0,0.0,0.0,2018
3,079601002,481.0,100.0,91.0,36.0,19.0,7.0,0.0,89.0,28.0,...,1.0,38.0,9.0,1.0,1.0,10.0,11.0,0.0,1.0,2018
4,079601003,884.0,154.0,179.0,74.0,40.0,13.0,2.0,141.0,77.0,...,3.0,54.0,18.0,4.0,0.0,23.0,30.0,0.0,1.0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7390,079621029,514.0,29.0,26.0,6.0,7.0,17.0,7.0,41.0,23.0,...,10.0,25.0,49.0,10.0,0.0,14.0,18.0,0.0,0.0,2020
7391,079621030,561.0,35.0,59.0,27.0,8.0,17.0,5.0,61.0,50.0,...,4.0,44.0,43.0,5.0,0.0,13.0,17.0,0.0,0.0,2020
7392,079621031,988.0,106.0,96.0,17.0,9.0,37.0,7.0,156.0,40.0,...,7.0,33.0,32.0,7.0,1.0,15.0,28.0,1.0,0.0,2020
7393,079621032,683.0,71.0,65.0,16.0,8.0,31.0,17.0,105.0,27.0,...,7.0,23.0,30.0,6.0,0.0,8.0,7.0,1.0,0.0,2020


### ETL Nationalities

In [145]:
# extract only those related to Madrid municipality
con = duckdb.connect('../nationalities.duckdb')
cleanNatTable = con.execute(f'SELECT * FROM nationalitiesClean').df()
con.close()
cleanNatTable = cleanNatTable[cleanNatTable['Madrid_section'].str.contains('0796', regex=False)]

cleanNatTable = cleanNatTable.rename(columns = {'Habitantes':'suma de categorías de nacionalidad'})  #rename in right format

cleanNatTable['Madrid_section'] = cleanNatTable['Madrid_section'].str.replace(' ', '') # remove extra spaces

df_to_DBtable('integration.duckdb',cleanNatTable, 'nationalitiesClean_Madrid')


display(cleanNatTable)

,Afganistán,Albania,Alemania,Andorra,Angola,Antigua_y_Barbuda,Arabia_Saudita,Argelia,Argentina,Armenia,...,Uganda,Uruguay,Uzbekistán,Vanuatu,Venezuela,Vietnam,Year,Yemen,Zambia,Zimbabwe
1271,110.0,272.0,4645.0,18.0,216.0,2.0,247.0,1162.0,4933.0,561.0,...,17.0,901.0,64.0,0.0,19366.0,251.0,2018,33.0,2.0,18.0
1272,12.0,23.0,606.0,2.0,5.0,0.0,7.0,89.0,585.0,22.0,...,2.0,86.0,3.0,0.0,1447.0,22.0,2018,0.0,0.0,0.0
1273,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,6.0,0.0,2018,0.0,0.0,0.0
1274,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,3.0,0.0,0.0,9.0,0.0,2018,0.0,0.0,0.0
1275,0.0,0.0,5.0,0.0,0.0,0.0,0.0,2.0,15.0,0.0,...,0.0,5.0,0.0,0.0,21.0,0.0,2018,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13413,0.0,0.0,2.0,0.0,5.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,2020,0.0,0.0,0.0
13414,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,19.0,0.0,2020,0.0,0.0,0.0
13415,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,40.0,0.0,2020,0.0,0.0,0.0
13416,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,8.0,0.0,2020,0.0,0.0,0.0
